# Table of Content <a id='toc'></a>


&nbsp;&nbsp;&nbsp;&nbsp;[Module 7 - Biopython](#0)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Broad examples of what we can do with Biopython](#1)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Main concept of Biopython](#2)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Objects](#3)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Help - important!](#4)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[`Seq` Object and (Alphabets)](#5)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Indexing and slicing of `Seq`](#6)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Methods of `Seq` objects](#7)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Mutability of `Seq` objects](#8)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[deprecated - Alphabets](#9)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;['Biological' methods of `Seq` objects](#10)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Bio.SeqIO Module and SeqRecord Object](#11)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Reading sequence records from files](#12)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[`SeqRecord` objects](#13)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Slicing a SeqRecord](#14)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[`SeqFeature` object](#15)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Writing to sequence files](#16)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Accessing Online Databases](#17)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Now, you can try to solve the exercises.](#18)

&nbsp;&nbsp;&nbsp;&nbsp;[Additional Theory](#19)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[On `SeqRecord` iterators and processing large sequence files with `SeqIO`](#20)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[More on `SeqFeature` objects](#21)


[back to the toc](#toc)

<br>

# Module 7 - Biopython <a id='0'></a>
----------------------------------

<br>

Let's consider a typical task in bioinformatics: we are interested in finding the GC% for some sequences. For this we would need i) open the file, ii) parse and extract the sequence information, and iii) calculate and report their GC%. In a very simple example, we could write something like following: 


In [ ]:
my_seq = {}
identifier = None
with open("data/my_sequences.fa") as input_file:
    for line in input_file:
        if line[0] == ">":
            identifier, *description = line.strip()[1:].split(" ", 1)
            my_seq[identifier] = ""
        else:
            my_seq[identifier] += line.strip().upper()
for identifier in my_seq:
    gc_count = my_seq[identifier].count("G") + my_seq[identifier].count("C")
    gc_pc = gc_count / len(my_seq[identifier]) * 100
    print("GC% for {} is {:.2f}".format(identifier, gc_pc))

Now let's see how it would look like if we used Biopython:

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import GC
for rec in SeqIO.parse("data/my_sequences.fa", "fasta"):
    print("GC% for {} is {:.2f}".format(rec.id, GC(rec.seq)))

<br>


[back to the toc](#toc)

<br>

## Broad examples of what we can do with Biopython <a id='1'></a>

- Sequence analysis
  - Motif
  - Search: HMMs, Alignments
  - Restriction
- Structures
  - SCOP
  - PDB
- Database query
- Phylogeny
- Pathway
- And more ...


[back to the toc](#toc)

<br>

## Main concept of Biopython <a id='2'></a>

- I/O interface and parsing abilities for bioinformatics files/DBs
  - Blast
  - FASTA
  - PubMed
  - SwissProt
- Efficient and practical data-structures for bioinformatics data
 - Sequences
 - Alignments
 - Structures
- Methods implementing bioinformatics analysis
 - Translation
 - Classification
 - Phylogeny trees
- Interface to common bioinformatics programs
  - Standalone Blast
  - ClustalW
  - EMBOSS


[back to the toc](#toc)

<br>

## Objects <a id='3'></a>

In Biopython there are many modules and each module contains several major new data types. Objects created with these types serve specific purposes in the above mentioned functionalities. We will focus on sequence analysis; some new objects we will discover are:

- `Seq`
- `Alphabet`
- `SeqRecord`
- `SeqFeature`


[back to the toc](#toc)

<br>

## Help - important! <a id='4'></a>

- A relatively detailed tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html
- Help on certain concepts and modules: https://biopython.org/wiki/Category%3AWiki_Documentation

In this notebook we will cover basics of the Biopython package:
    - Seq object and alphabets
    - Bio.SeqIO module and SeqRecord object
    - Interacting with external DBs


[back to the toc](#toc)

<br>

## `Seq` Object and (Alphabets) <a id='5'></a>

Seq object is a flexible encapsulator for _sequence-like_ strings. It has two main components:
- A Python string representing a _biological sequence_
- An Alphabet object describing the _letters_ used in this sequence.

The related module is __Bio.Seq__

In [ ]:
from Bio.Seq import Seq

Let's create our first `Seq` object. The only required positional argument is _'data'_, which is used for the sequence string. 

In [ ]:
my_seq = Seq('AGCGCGATTTATATATAGCGAGCGATTCGGAGCGATCGACGGATTCGAC')

In [ ]:
my_seq


[back to the toc](#toc)

<br>

### Indexing and slicing of `Seq` <a id='6'></a>

A `Seq` object is based on Python's `str` object, so most methods we can use with a `str` object, we can also use with a `Seq` object. Let's try to use indexing and slicing with our sequence:

In [ ]:
print("5th nucleotide is {}".format(my_seq[6]))

In [ ]:
my_seq[5:11], my_seq[16:22]

> Note that the result of slicing is a **new** `Seq` object!

It is important to make a difference between indexing and single-element slicing. Let's have a *closer* look at the results of these two operations:

In [ ]:
# Using index
my_seq[6]

In [ ]:
# And using slice
my_seq[6:7]

#### Question 1.

Can you make a new `Seq` object combining the nucleotides 6-11 and 17-22. *Hint:* try to use slices

In [ ]:
# You can use this cell for Question 1.



[back to the toc](#toc)

<br>

### Methods of `Seq` objects <a id='7'></a>

As `Seq` objects inherits many properties from `str`, many of its methods are available to us to use on our sequences.

In [ ]:
my_seq.count('TAT') # counts the occurences of the input

In [ ]:
my_seq.lower()

In [ ]:
my_seq.index('TAT') # gives the index (0-based) of the first occurence of the input

Notice that the main difference from calling these methods on a `str` object is that they return new `Seq` objects as a result:

In [ ]:
my_seq.split('GAT')

Of course, being a specialized object, it **does not** support all `str` methods. And more importantly, it implements many **new** methods that relate to biological sequences. Let's explore some examples for both cases:

In [ ]:
# .isdigit() is one of the string methods that is not supported by Seq objects

my_seq.isdigit()

In [ ]:
# on the other hand, .count_overlap() is one the new methods that Seq implements

my_seq.count_overlap('TAT')

In [ ]:
import Bio
Bio.__version__

#### Question 2.
Can you tell the difference between the `str` method `.count()` and the specialized `Seq` method `.count_overlap()`?

In [ ]:
# You can use this cell for Question 2.


In [ ]:
# another example for Seq specific method is .reverse_complement()

print("Reverse complement of my sequence is: {}".format(my_seq.reverse_complement()))

> To see all methods and their descriptions you can always visit the documentation pages or simply use the help function whenever needed: `help(Seq)`.


[back to the toc](#toc)

<br>

### Mutability of `Seq` objects <a id='8'></a>

Just like the `str` object, the `Seq` object does not support item assignment; in other words, it is __immutable__.

In [ ]:
my_seq[6] = 'T'

However, in biology, **mutations are fundamental**. An important variant of the `Seq` object is the `MutableSeq` object. There are two ways of creating a `MutableSeq` object. We can convert our normal `Seq` object into a mutable one:

In [ ]:
mutable_seq = my_seq.tomutable()

Or, we can import the `MutableSeq` class and instantiate ourselves:

In [ ]:
from Bio.Seq import MutableSeq
mutable_seq = MutableSeq('AGCGCGATTTATATATAGCGAGCGATTCGGAGCGATCGACGGATTCGAC')

In [ ]:
mutable_seq[6] = 'T'
mutable_seq

#### Question 3.
Is point mutation the only type of mutation supported by the `MutableSeq` object? Can we introduce insertions/deletions by simple item assignment? Try it!

In [ ]:
# You can use this cell for Question 3.



[back to the toc](#toc)

<br>

### deprecated - Alphabets <a id='9'></a>

In september 2020 (v1.78), Biopython removed the `Bio.Alphabet` module.
The purpose of the `Alphabet` class was to provide a consistent data structure and functionality around genetic code contained in different molecule types, such as DNA, RNA, protein.

However experience proved that such a feature was, not worth it. 

If you play with older code, or have to work with an older version of the module,
here is a little [guide](https://biopython.org/wiki/Alphabet) to help you deal with the `Alphabet`.



[back to the toc](#toc)

<br>

### 'Biological' methods of `Seq` objects <a id='10'></a>
Let's now focus on `Seq` specific 'biological' methods:

- complement()
- reverse_complement()
- transcribe()
- back_transcribe()
- translate()

These methods do not *require* any other extra arguments and they return a new `Seq` object.

#### Complementing and reverse complementing sequences

In [ ]:
intronless_dna = Seq(
    'CACCTCTGGAGCGGACTTATTTACCAAGCATTGGAGGAATATCGTAGGTAAAAATGCCTA'
    'TAGGATCCAAAGAGAGGCCAACATTTTTTGAAATTTTTAAGACACGCTGCAACAAAGCA')
print("intronless_dna:", intronless_dna)


# If we need to find its complement sequence:

compl_seq = intronless_dna.complement()
print()
print("compl_seq:", compl_seq)

# If we would need to find the reverse complement of our sequence, we can simply

rev_strand = intronless_dna.reverse_complement()
print()
print("rev_strand:", rev_strand)
print("alternatively:", compl_seq[::-1])

#### Transcription and reverse transcription of sequences

In [ ]:
# Let's now transcribe this piece of DNA into RNA. This is an intronless DNA sequence, so

rna_seq = intronless_dna.transcribe()
print("RNA:", rna_seq)

In [ ]:
# We can also reverse transcribe an RNA sequence to cDNA...
cdna_seq = rna_seq.back_transcribe()
cdna_seq

#### Translation of sequences

Up to here, we have described an intronless DNA sequence, and 'transcribed' it into an RNA sequence, `rna_seq`. If we know where the CDS starts, we can also translate the CDS into a protein sequence. In this example, the start codon can be found at 53. position.

In [ ]:
cds_seq = rna_seq[53:]
cds_seq

In [ ]:
protein_seq = cds_seq.translate()
protein_seq

The `.translate()` method can take extra arguments to finetune the translation specifics:

- **table** - Which codon table to use? Tables are based on [NCBI's Genetic Codes](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi). Default is the "Standard" table.


- **stop_symbol** - Single character string, what to use for terminators.  Default is the asterisk, *


- **to_stop** - Boolean, defaults to False meaning do a full                                                        translation continuing on past any stop codons (translated as the                                                  specified stop_symbol).  If True, translation is terminated at                                                      the first in frame stop codon (and the stop_symbol is not                                                          appended to the returned protein sequence).


- **cds** - Boolean, indicates this is a complete CDS.  If True,                                                    this checks the sequence starts with a valid alternative start                                                      codon (which will be translated as methionine, M), that the                                                        sequence length is a multiple of three, and that there is a                                                        single in frame stop codon at the end (this will be excluded                                                        from the protein sequence, regardless of the to_stop option).                                                      If these tests fail, an exception is raised.


- **gap** - Single character string to denote symbol used for gaps. Defaults to the minus sign.

In [ ]:
# Codon tables can be defined by string names (NCBI genetic code table name)
# or integers (NCBI table ids)
codon_tables = [
    "Vertebrate Mitochondrial", # 2. The Vertebrate Mitochondrial Code
    6                           # 6. The Ciliate, Dasycladacean and Hexamita Nuclear Code
]

# Stop symbols could be and single character string
stop_symbols = ["\\", "£"]  # Careful with "\", we need to escape with a backslash "\"

# to_stop is boolean, either True or False
to_stops = [True, False]

print("{: <24}\t{}\t{}\t{}".format("Table", "Symbol", "to_stop", "peptide"))
for table in codon_tables:
    for stop_symbol in stop_symbols:
        for to_stop in to_stops:
            protein = cds_seq.translate(table=table,
                                        stop_symbol=stop_symbol,
                                        to_stop=to_stop)
            print("{: <24}\t{}\t{}\t{}".format(table, stop_symbol, to_stop, protein))

In [ ]:
# Optional, more advanced code

# The nested for-loops in the above example can be combined efficiently using
# the 'itertools' module. The 'product' method returns the cartesian product of
# its arguments, equivalent to a nested for-loop

import itertools
header = ["Table", "Symbol", "to_stop", "peptide"]
row = "{: <24}\t{}\t{}\t{}"

print(row.format(*header)) # (*) unpacks the list as arguments to the function!
for table, stop_symbol, to_stop in itertools.product(codon_tables, stop_symbols, to_stops):
    protein = cds_seq.translate(table=table,
                                stop_symbol=stop_symbol,
                                to_stop=to_stop)
    print(row.format(table, stop_symbol, to_stop, protein))

#### Where alphabets were useful

A potential source of error may come from the fact that two `Seq` objects may represent different types of biological sequences (protein and cDNA for instance).

Indeed, nothing prevents us from erroneously applying `Seq` methods to types of sequences that would 'biologically' not support these actions. For example, if we try to reverse-complement a protein sequence:

In [ ]:
protein_seq.reverse_complement() # returns a meaningless sequence by trying to interpret AA as IUPAC nts.

Or, we can concatenate two incompatible `Seq` objects together:

In [ ]:
print( cds_seq + protein_seq )


[back to the toc](#toc)

<br>

## Bio.SeqIO Module and SeqRecord Object <a id='11'></a>

The `Bio.SeqIO` module provides a simple and uniform interface to read and parse from and write to various sequence file formats (including multiple sequence alignments). The `Bio.SeqIO` module supports a large number of sequence file formats, including *fasta*, *fastq*, and *genbank* and many more, which can be found [here](https://biopython.org/wiki/SeqIO). All sequences in this method will be accessed as `SeqRecord` objects.


[back to the toc](#toc)

<br>

### Reading sequence records from files <a id='12'></a>

The main function of the module is `.parse()`, which takes a file handle (or filename) and format name, and returns a `SeqRecord` iterator. An iterator is an object that can be iterated upon, meaning that you can traverse through all the values, one by one. In this case `Bio.SeqIO.parse()` method returns an iterator of `SeqRecord` objects extracted and parsed from the input file. We can then iterate over the records and process them in a very efficient manner.

In [ ]:
# Let's parse an example fasta file and iterate over its records

from Bio import SeqIO
for record in SeqIO.parse("data/example.fa", "fasta"):   # the for loop iterates over the SeqRecord objects
    print(record.id)                                # each SeqRecord object contains an ID

Sometimes, it could be handier to supply a filehandle instead of the filename to `.parse()` method. This also permits the use of different input resources such as streams. We can re-examine the previous example:

In [ ]:
with open("data/example.fa", "r") as filehandle:
    for rec in SeqIO.parse(filehandle, "fasta"):
        print(rec.id)

The second argument of the `Bio.SeqIO.parse()` method defines the type of the input file and it is mandatory. A detailed list of possible values for this argument can be found on [the official documentation](https://biopython.org/wiki/SeqIO). Let's open another file, **example.gp**, that contains the same sequences as before but this time in GenBank format (full GenPept format to be more precise).

In [ ]:
for rec in SeqIO.parse("data/example.gp", "genbank"):
    print(rec.id)


[back to the toc](#toc)

<br>

### `SeqRecord` objects <a id='13'></a>
We have mentioned that the results of 'Bio.SeqIO' parsing methods are `SeqRecord` objects. We have already seen that these objects contain an `.id` attribute. A `SeqRecord` object holds, beside the sequence (as a `Seq` object), identifiers (ID and name), description and optionally annotation and sub-features. The completeness of a record will depend on its source.

`SeqRecord` contains many attributes, some of which can be rather complex. We will further examine most commonly used ones:

- seq
- id
- name
- description
- annotations
- features

In [ ]:
# Let's grab the last record in our example file to use in our following cells
records = list(SeqIO.parse("data/example.gp", "genbank"))
rec = records[-1]


In [ ]:
# rec.seq is a `Seq` object containing the sequence and the alphabet
rec.seq

In [ ]:
# rec.id is a string containing the sequence identifier
rec.id

In [ ]:
# rec.name is a string containing the display name of the sequence
# it may be identical to or quite different than rec.id !!
rec.name

In [ ]:
# rec.description is also a simple string
rec.description

In [ ]:
# rec.annotations is dictionary that contains different annotations about the sequence record
# When created from a GenBank source, certain keys can be expected to be present such as source, taxonomy
print("record is from", rec.annotations["source"])
print("and its full taxonomy is", rec.annotations["taxonomy"])

In [ ]:
# rec.features is a list
# It contains SeqFeature objects which we will discuss briefly later.
# But in a nutshell, they hold information about various positional features identified on the sequence

rec.features[-1]

Let's examine the records contained in 'example.gb' file more closely.

In [ ]:
for index, rec in enumerate(SeqIO.parse("data/example.gp", "genbank")):
    print("{} [{}]\t{}\tlength {}\twith {} features and {} annotations".format(
        index, rec.id, rec.description[:21], len(rec.seq), len(rec.features), len(rec.annotations)))

#### Question 4.

Similarly parse the **example.fa** file and compare the `SeqRecord` objects to the above cell from **example.gp**. Which attributes are identical? What are the differences? Why?

In [ ]:
# You can use this cell for Question 4.


We can create a `SeqRecord` directly. We need to import the class from its module, `Bio.SeqRecord`, first. In order to create its sequence we will also need the `Bio.Seq.Seq` or a similar `Seq` class.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

record = SeqRecord(Seq("ACGGCTATCTGAGGACTACGAGCATCATCGA"),
                   id="my_seq_ID", name="made-up sequence",
                   description="Just some randomly typed ATGCs")
print(record)


[back to the toc](#toc)

<br>

### Slicing a SeqRecord <a id='14'></a>

A very useful feature of the `SeqRecord` object is that we can simply slice it just like slicing a `str` or `list`. Within the context of a list, we refer to slice positions as indices. In this case, they correspond to positions along the sequence. Just like the `Seq` object.

We have to keep in mind that the positions are zero-based, that is a portion of the sequnce that maps to 10-50 bases would need to be coded as 9:50

In [ ]:
# Let's grab the last record in our example file
records = list(SeqIO.parse("data/example.gp", "genbank"))
last_record = records[-1]

# We were told that the zinc binding site is from 48 to 178 (do not forget to convert to 0-based)
# If you are interested in finding out by examining the features, see below "Optional Material"
zinc_binding = last_record[47:178]
print(zinc_binding)

When a `SeqRecord` is sliced:

- A new `SeqRecord` is returned
- The sequence is sliced but the id, name and description are kept unchanged
- Only the features fall **completely** wihtin the new coordinates are kept (see Optional section for more information)
- Positions of any feature inherited will be adjusted to the new coordinates
- All positions are 0-based 

#### Question 5.

Can you create a new sequence record containing the nucleotides 23-123 from the `last_record`?

In [ ]:
# You can use this cell for Question 5.



[back to the toc](#toc)

<br>

### `SeqFeature` object <a id='15'></a>


We have seen that the `SeqRecord` object has an attribute called `.features` which is a list of `SeqFeature` objects. Let's understand this object a little bit better.

`SeqFeature` object describes a feature along a sequence. Some of its useful attributes are:

- `.location` which tells us the 0-based coordinates of the feature
- `.type` which holds the type or the annotation of the feature
- `.qualifiers` is an ordered dictionary (the keys are always returned in same order) that holds qualifying information about the feature. These are analogous to the qualifiers from a GenBank feature table.

`SeqFeature` is defined within its own module: `Bio.SeqFeature`. For more detail on this object and the module, you can start with its [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc37) and [documentation](https://biopython.org/docs/1.74/api/Bio.SeqFeature.html).

Let's dissect one `SeqFeature` object:

In [ ]:
# Let's grab the third record in our example file
rec = list(SeqIO.parse("data/example.gp", "genbank"))[2]

# Simply print features neatly 
for f_i, feature in enumerate(rec.features):
    print("Feature index:", f_i)
    print(feature)

We observe:

1) `.type` attribute contains useful information for selecting the right kind of feature we might be interested in. Its values are controlled - any information that is available in the database will be used. But usually databases such as NCBI and UniProt are vocabulary controlled and use always the same keywords such as 'source' and 'CDS'.

2) `.qualifiers` has no strict structure - it is a `dict` that contains many relevant information about the `.feature`. The **only consensus** is that the values are always a list! We have to explore and figure out what we need/want from `.qualifiers` ourselves.

Let's see how we access the data within a `SeqFeature` object:

In [ ]:
# continuing with the last feature object:

# be careful value of location is not list, tuple etc
print("its location:", feature.location)
print("its start:", feature.location.start)
print("and end:", feature.location.end)
print()
print("its type:", feature.type)
print()
print("its qualifiers:", feature.qualifiers) # this is an OrderedDict, essentially a dict

#### Question 6.

Can you print which translation table (only integer code) should be used for translating this CDS sequence?

In [ ]:
# You can use this cell for Question 6.



[back to the toc](#toc)

<br>

### Writing to sequence files <a id='16'></a>
So far we have seen how to parse sequence files and work with sequence records. In many applications, we will also need to write our processed sequence records back into a standard sequence file. For this purpose, the `Bio.SeqIO` module has a `.write()` function. It can be thought as the reverse of the `.parse()` method we have learnt above.

In [ ]:
# Let's create 10 random sequences each around 400 bases long. For this we can use the 'random' module

import random

records = []
for i in range(10):
    random_atgc = random.choices("ATGC", k=random.randint(380, 420))
    record = SeqRecord(Seq("".join(random_atgc)),
                       id="my_seq_{}".format(i),
                       name="random sequnce {}".format(i),
                       description="A randomly generated sequence")
    records.append(record)
SeqIO.write(records, "random_sequences.fa", "fasta")


[back to the toc](#toc)

<br>

## Accessing Online Databases <a id='17'></a>

Biophyton introduces multiple means to interact with commonly used bioinformatics databases over the internet. We will cover the basics of accessing online databases with two databases:

1. NCBI's Entrez
2. UniProt/SwissProt's ExPASy

The Entrez can be queried using the `Bio.Entrez` module. There many different functionalities, but today we will only cover the `.efetch()` method, which we can use to fetch records from Entrez over the internet.

Let's discover this functionality with an example.

In [ ]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "my.address@email.ch"  # This is obligatory.You should use yours!

with Entrez.efetch( db="protein",                     # which Entrez database to use, nucleotide, protein,..
                    rettype="gp",                     # return-type: fasta, gb (genbank), gp (genpept)
                    retmode="text",                   # return-mode: text or xml (perhaps other options?)
                    id="NP_001315822.1" ) as handle:  # the accession ID of the record we would like to fetch
    # Once it is a handle, we know the rest from SeqIO...
    seq_record = SeqIO.read(handle, "gb")
    
print("{} with {} features".format(seq_record.id, len(seq_record.features)))

In [ ]:
with Entrez.efetch( db="protein",
                    rettype="fasta",                 # if the return-type is set to 'fasta'
                    retmode="text",
                    id="NP_001315822.1" ) as handle:
    seq_record = SeqIO.read(handle, "fasta") # we can only parse it as fasta
    
print("{} with {} features".format(seq_record.id, len(seq_record.features)))

The `Bio.ExPASy` module provides functions to interact with the UniProt/SwissProt database. 

In [ ]:
from Bio import ExPASy
from Bio import SeqIO

with ExPASy.get_sprot_raw("P48977") as handle:   # a simple .get_sprot_raw() function fetches records
    record = SeqIO.read(handle, "swiss")         # which we can then parse with SeqIO
print("Retrived {}: {}".format(record.id, record.description))


[back to the toc](#toc)

<br>

### Now, you can try to solve the exercises. <a id='18'></a>




<br>
<br>



[back to the toc](#toc)

<br>

# Additional Theory <a id='19'></a>
-----------------------------

The Biopython module is 'huge'! Under this section, we have included some more information on basic functionality which is not needed for the exercises.


[back to the toc](#toc)

<br>

### On `SeqRecord` iterators and processing large sequence files with `SeqIO` <a id='20'></a>

We have learnt how we can iterate over the sequence records contained in an input file using the `.parse()` method of `SeqIO`. There are some important limitations that come with iterators. Below, we will explore some of there limitations, how we can overcome them but at a cost in memory usage. Finally, some alternatives if we need to process large files without iterators.

In [ ]:
from Bio import SeqIO

In [ ]:
# A word about iterators. Once we iterate over all values an iterator gets empty.
# That means, we can not access the contents any longer.

# Let's create an iterator of sequence records:
records = SeqIO.parse("data/example.fa", "fasta")

# We can access the next item (in this case, the first one) via core next() method:
print("First record is", next(records).id)

# Now, let's iterate over all values with a for loop:
for record in records:
    print("first use", record.seq[:5])
print("We have reached the end of the iterator")

# At this point, we have reached the end of the iterator. We can not use it again:
for record in records:
    print("second use", record.seq[:5])

In [ ]:
# If we force the iterator with next(), it will not return a None, but a StopIteration exception
next(records)

Therefore, sometimes it is useful to convert the iterator into a list or a dictionary, given that it is not too big to fit into the memory. Iterators can be easily converted into lists, simply casting a `list()` function. For conversion into a dictionary, the `Bio.SeqIO` provides a specialized method called `.to_dict()`. 

In [ ]:
# Converting the sequence record iterator into a list

records = list(SeqIO.parse("data/example.fa", "fasta"))
# Now, let's loop over all values with for:
for record in records:
    print("first use", record.seq[:5])
# And again:
for record in records:
    print("second use", record.seq[:5])

In [ ]:
# Converting the sequence record iterator into a dictionary

records = SeqIO.parse("data/example.fa", "fasta")
rec_dict = SeqIO.to_dict(records)
print("Listing all items:")
for rec_id, rec in rec_dict.items():
    print(rec_id, rec.seq[:5])
print()

specific_rec_id = "ADY55933.1"
print("Accessing to a specific record:", specific_rec_id)
print(specific_rec_id, rec_dict[specific_rec_id].seq[:5])

For larger files, it isn’t possible to hold everything in memory, so `Bio.SeqIO.to_dict()` is not suitable. In these situations we can use the `Bio.SeqIO.index()` function. This will not populate a dictionary, rather index the input file such that we can access records in a an arbitrary order. This should be used with very large files; for small files it will be slower than the `.to_dict()` method.

In [ ]:
rec_dict = SeqIO.index("data/example.fa", "fasta")
print(specific_rec_id, rec_dict[specific_rec_id].seq[:5])


[back to the toc](#toc)

<br>

### More on `SeqFeature` objects <a id='21'></a>


`SeqFeature` object describes a feature along a sequence. Some of its useful attributes are:

- `.location` which tells us the 0-based coordinates of the feature
- `.type` which holds the type or the annotation of the feature
- `.qualifiers` is an ordered dictionary (the keys are always returned in same order) that holds qualifying information about the feature. These are analogous to the qualifiers from a GenBank feature table.

`SeqFeature` is defined within its own module: `Bio.SeqFeature`. For more detail on this object and the module, you can start with its [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc37) and [documentation](https://biopython.org/docs/1.74/api/Bio.SeqFeature.html).

Let's extract some useful information for our sequence record from its features.

In [ ]:
for feature in rec.features:
    print("From {} to {}, there is a '{}' feature".format(
        feature.location.start, feature.location.end, feature.type))
    for key, val in feature.qualifiers.items():
        print("    {} is {}".format(key, val))

When we slice a `SeqRecord` that contains `SeqFeature`s, we have learnt that only those features that completely fall within the slice will be kept. Then the positions will be offsetted relative to the new sliced sequence.

In [ ]:
# Let's grab the last record in our example file
records = list(SeqIO.parse("data/example.gp", "genbank"))
last_record = records[-1]

# We have seen above that the zinc binding site is from 47 to 178 (0-based)
zinc_binding = last_record[47:178]
incomplete_overlap = last_record[30:100]
print(zinc_binding)
print()

# Let's have a closer look at the single feature included within the sliced portion
feat = zinc_binding.features[0]
print("The feature is of '{}' type".format(feat.type))
print("It is from {} to {}".format(feat.location.start, feat.location.end))
print("Its location is {}".format(feat.location)) # We have the positions of 4 residues that make contact with Zn
print("Its qualifiers are: {}".format(feat.qualifiers))
print()

# On the other hand, the incompletely overlapping part (look at the features)
print(incomplete_overlap)